In [0]:
dbutils.widgets.text("catalog", "hive_metastore")
dbutils.widgets.text("schema", "delta_nip")
dbutils.widgets.text("date", "04052025")
dbutils.widgets.text("root_location", "/user/hive/warehouse/delta_nip.db/")

In [0]:
%sh

# ls -l /Workspace/Users/jn@nipgroup.com/ventiv_ddl/small_load
ls -l /dbfs/mnt/
ls -l /dbfs/mnt/load
ls -l /dbfs/mnt/schema
ls -l /dbfs/mnt/sas
ls -l /dbfs/mnt/pgp

In [0]:
import os

directory = "/Workspace/Users/jn@nipgroup.com/ventiv_ddl/small_load/"
mnt_load = "/dbfs/mnt/load"
# List files in the remote directory
files = os.listdir(directory)
print(files)

for file in files:
    if file.endswith(".csv"):
        destination_path = os.path.join(mnt_load, file)
        source_path = os.path.join(directory, file) # Remote file path

        # Download the file to DBFS temp directory with the same name as the remote file
        with open(source_path, "rb") as source_file, open(destination_path, "wb") as destination_file:
            while True:
                data = source_file.read()
                if not data:
                    break
                destination_file.write(data)
                
            print(f"File {file} downloaded successfully.")

In [0]:
df = spark.read.csv("/mnt/load/ref_class_code_exported_data_04052025.csv", header=True, inferSchema=True)

# Step 2: Extract the first row (original header) as a DataFrame
header_row = df.limit(1)
header_row.show()

# # Step 3: Get the remaining rows
# data_rows = df.subtract(header_row)

# # Step 4: Combine the header row as the second row in the DataFrame
# combined_df = data_rows.union(header_row)

# # Step 5: Optionally assign default column names
# df_import_01 = combined_df.toDF(*[f"_c{i}" for i in range(len(combined_df.columns))])

In [0]:
existing_schema = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/ref_class_code").schema
existing_schema.simpleString()

print(existing_schema)

In [0]:
existing_schema = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/claims").schema

print(existing_schema)

In [0]:
# Step 1: Get the schema of the delta table
table_schema = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/ref_class_code").schema

print(existing_schema)

# Step 2: Load 
df = spark.read.csv("/mnt/load/ref_class_code_exported_data_04052025.csv", header=False, inferSchema=True)

# 
df.show()

# # Step 2: Extract the first row (original header) as a DataFrame
# header_row = df.limit(1)
# header_row.show()

# # Step 3: Get the remaining rows
# data_rows = df.subtract(header_row)

# # Step 4: Combine the header row as the second row in the DataFrame
# combined_df = data_rows.union(header_row)

# # Step 5: Optionally assign default column names
# df_import_01 = combined_df.toDF(*[f"_c{i}" for i in range(len(combined_df.columns))])

In [0]:
root = dbutils.widgets.get("root_location")
table_name = "ref_class_code"
print(f"{root}{table_name}")

In [0]:

root = dbutils.widgets.get("root_location")
table_name = "ref_class_code"
table_full_path = f"{root}{table_name}"

# Step 1; Read schema from existing Delta table
existing_table_schema = spark.read.format("delta").load(table_full_path).schema
# print(existing_table)
column_names = existing_table_schema.fieldNames()

# print(column_names)

# Step 2: Load
new = spark.read.csv("/mnt/load/ref_class_code_no_header.csv", header=False)

#
# print("initial load with schema")
new.schema
# df.display()

######## APPROACH 2
for i, name in enumerate(column_names):
    df = df.withColumnRenamed(f"_c{i}", name)

# print("after rename")
# df.schema
# df.display()

######## APPROACH 1

# # Step 3: Extract the first row (original header) as a DataFrame
# header_row = df.limit(1)

# # Step 4: Get the remaining rows
# data_rows = df.subtract(header_row)

# # Step 4: Combine the header row as the second row in the DataFrame
# combined_df = data_rows.union(header_row)

# # Step 5: Optionally assign default column names
# df_import_01 = combined_df.toDF(*[f"_c{i}" for i in range(len(combined_df.columns))])
# df_import_01.display()
# # Apply to new data without headers
# input_df = spark.read.format("csv") \
#   .option("header", "false") \
#   .load("/path/to/input.csv")

# # Rename the default columns to match your schema
# for i, name in enumerate(column_names):
#     input_df = input_df.withColumnRenamed(f"_c{i}", name)

# # Apply data types from schema
# for field in existing_schema.fields:
#     input_df = input_df.withColumn(field.name, 
#                                    input_df[field.name].cast(field.dataType))

In [0]:

root = dbutils.widgets.get("root_location")
table_name = "sp_premium_triangles"
table_full_path = f"{root}{table_name}"

# Step 1; Read schema from existing Delta table
existing_table_schema = spark.read.format("delta").load(table_full_path).schema
# print(existing_table)
print(existing_table_schema)
column_names = existing_table_schema.fieldNames()

# print(column_names)

# Step 2: Load
triangle = spark.read.csv("/mnt/load/sp_premium_triangles_exported_data_04052025.csv", header=False)

#
# print("initial load with schema")
triangle.schema
# df.display()

triangle_withSchema = spark.read.csv("/mnt/load/sp_premium_triangles_exported_data_04052025.csv", header=False, schema=existing_table_schema)

#
# print("initial load with schema")
triangle_withSchema.schema
# df.display()

# ######## APPROACH 2
# for i, name in enumerate(column_names):
#     df = df.withColumnRenamed(f"_c{i}", name)

# print("after rename")
# df.schema
# df.display()

######## APPROACH 1

# # Step 3: Extract the first row (original header) as a DataFrame
# header_row = df.limit(1)

# # Step 4: Get the remaining rows
# data_rows = df.subtract(header_row)

# # Step 4: Combine the header row as the second row in the DataFrame
# combined_df = data_rows.union(header_row)

# # Step 5: Optionally assign default column names
# df_import_01 = combined_df.toDF(*[f"_c{i}" for i in range(len(combined_df.columns))])
# df_import_01.display()
# # Apply to new data without headers
# input_df = spark.read.format("csv") \
#   .option("header", "false") \
#   .load("/path/to/input.csv")

# # Rename the default columns to match your schema
# for i, name in enumerate(column_names):
#     input_df = input_df.withColumnRenamed(f"_c{i}", name)

# # Apply data types from schema
# for field in existing_schema.fields:
#     input_df = input_df.withColumn(field.name, 
#                                    input_df[field.name].cast(field.dataType))

In [0]:

root = dbutils.widgets.get("root_location")
table_name = "sp_premium_triangles"
table_full_path = f"{root}{table_name}"

# Step 1; Read schema from existing Delta table
existing_table_schema = spark.read.format("delta").load(table_full_path).schema
# print(existing_table_schema)
column_names = existing_table_schema.fieldNames()

# print(column_names)

# Step 2: Load
df = spark.read.csv("/mnt/load/sp_premium_triangles_exported_data_04052025.csv", header=False)

# df.display()
count = len(df.schema.fieldNames())
print(f"no schema: {count}")

print(f"with schema: {len(existing_table_schema.fieldNames())}")


# Validation
if len(column_names) == len(df.schema.fieldNames()):
    print("Schema match")
    # Rename the default columns to match your schema
    for i, name in enumerate(column_names):
        df = df.withColumnRenamed(f"_c{i}", name)
    
    print("pre data type enforcement")
    print(df.schema)

    # Apply data types from schema
    for field in existing_table_schema.fields:
        df = df.withColumn(field.name, df[field.name].cast(field.dataType))

    print("post data type enforcement")
    print(df.schema)
else:
    print("Schema mismatch")
    

# print("after rename")
# df.schema
# df.display()

In [0]:
from pyspark.sql.types import *
import pandas as pd

# Read the schema metadata CSV into a pandas DataFrame
schema_path = "/dbfs/mnt/schema/claims_import_01_schema.csv"  # Adjust this path
schema_pd = pd.read_csv(schema_path)

# Function to convert SQL types to Spark types with precision/scale
def sql_to_spark_type(row):
    sql_type = row["DATA_TYPE"].lower()
    is_nullable = row["IS_NULLABLE"] == "YES"
    
    # Handle different data types with their specific attributes
    if sql_type in ["decimal", "numeric"]:
        precision = row["NUMERIC_PRECISION"] if not pd.isna(row["NUMERIC_PRECISION"]) else 38
        scale = row["NUMERIC_SCALE"] if not pd.isna(row["NUMERIC_SCALE"]) else 0
        return DecimalType(precision, scale)
    elif sql_type in ["varchar", "nvarchar", "char", "nchar"]:
        # If length is specified use it, otherwise use a reasonable default
        length = row["CHARACTER_MAXIMUM_LENGTH"] 
        if pd.isna(length) or length == -1:  # -1 often means MAX
            return StringType()
        else:
            return StringType()  # Spark doesn't enforce string length
    
    # Map other types
    type_map = {
        "int": IntegerType(),
        "bigint": LongType(),
        "smallint": ShortType(),
        "tinyint": ByteType(),
        "bit": BooleanType(),
        "money": DecimalType(19, 4),
        "float": DoubleType(),
        "real": FloatType(),
        "datetime": TimestampType(),
        "datetime2": TimestampType(),
        "date": DateType(),
        "time": TimestampType(),
        "text": StringType(),
        "ntext": StringType(),
        "binary": BinaryType(),
        "varbinary": BinaryType(),
        "uniqueidentifier": StringType()
    }
    
    return type_map.get(sql_type, StringType())

# Build Spark schema based on SQL Server metadata
schema_fields = []
for _, row in schema_pd.iterrows():
    is_nullable = row["IS_NULLABLE"] == "YES"
    field = StructField(
        row["COLUMN_NAME"], 
        sql_to_spark_type(row), 
        nullable=is_nullable
    )
    schema_fields.append(field)

# Create the Spark schema
spark_schema = StructType(schema_fields)
# spark_schema.simpleString()
print(spark_schema)

In [0]:
from datetime import datetime

excel_file_path = "/mnt/sas/I2I_CLAIM_NIP_20240930.CSV"
date_string = excel_file_path.split('/')[-1].split('_')[-1].split('.')[0]
as_of_date = datetime.strptime(date_string, '%Y%m%d').strftime('%Y-%m-%d')
display(as_of_date)

df_import_01 = spark.read.csv(excel_file_path, header=True, inferSchema=False, schema=spark_schema)
df_import_01.createOrReplaceTempView("df_import_01")

delta_ref_org = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/ref_organization")
delta_ref_org.createOrReplaceTempView("delta_ref_org")

delta_ref_lob = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/ref_lob")
delta_ref_lob.createOrReplaceTempView("delta_ref_lob")

delta_ref_cov_desc = spark.read.format("delta").load("/user/hive/warehouse/delta_nip.db/ref_cov_desc")
delta_ref_cov_desc.createOrReplaceTempView("delta_ref_cov_desc")                                               

# joined_sql_df = spark.sql("""
#     SELECT 
#         c.*, 
#         d.column1, 
#         d.column2
#     FROM 
#         csv_data c
#     JOIN 
#         delta_data d
#     ON 
#         c.join_column = d.join_column
#     WHERE 
#         c.some_condition = 'value'
# """)

df_import_02 = spark.sql(f"""
    SELECT CAST('{as_of_date}' AS DATE) AS as_of,
        b.book,
        b.account_name,
        b.specialty_program,
        b.carrier,	
        c.lob,
        a.*,
        CAST(SUBSTRING(a.claim_last_updated_date_char, 1, 9) AS DATE) AS claim_last_updated_date,
        d.cov_desc,
        d.cov_desc_detail
    FROM df_import_01 AS a
        LEFT JOIN delta_ref_org AS b ON
            a.organization = b.organization
        LEFT JOIN delta_ref_lob AS c ON
            a.coverage = c.coverage
        LEFT JOIN delta_ref_cov_desc AS d ON
            b.book = d.book AND
            c.lob = d.lob AND
            a.tpa = d.tpa AND
            a.descriptors_source_coverage = d.descriptors_source_coverage;
""")

df_import_02.display()

# df_import_02_schema = spark.createDataFrame(df_import_02.rdd, schema=existing_table_schema)


In [0]:
from datetime import datetime
import calendar
from dateutil.relativedelta import relativedelta

# Parameter for how many months to go back
time_machine = 1  # Example: go back 9 months

# Get current date
current_date = datetime.now()

# Calculate target date by subtracting months (handles year transitions properly)
target_date = current_date - relativedelta(months=time_machine)

# Extract month and year from target date
target_month = target_date.month
target_year = target_date.year
month_name = calendar.month_name[target_month]
print(month_name)  # Using print instead of display for more universal compatibility

# Get the last day of the target month
last_day = calendar.monthrange(target_year, target_month)[1]

# Format the date strings
date_suffix = f"{target_year:04d}{target_month:02d}{last_day:02d}"
as_of = datetime.strptime(date_suffix, '%Y%m%d').strftime('%Y-%m-%d')

print(f"Date suffix: {date_suffix}")
print(f"As of date: {as_of}")

In [0]:
map = {
    "claims": f"I2I_CLAIM_NIP_{date_suffix}.CSV",
    "exposures": f"I2I_EXPOSURE_NIP_{date_suffix}.CSV",
    "organization": f"I2I_ORGANIZATION_NIP_{date_suffix}.CSV",
    "policy": f"I2I_POLICY_NIP_{date_suffix}.CSV",
    "premium": f"I2I_PREMIUM_NIP_{date_suffix}.CSV",
    "claims_jif": f"I2I_JIF_CLAIM_NIP_{date_suffix}.CSV",
    "exposures_jif": f"I2I_JIF_EXPOSURE_NIP_{date_suffix}.CSV",
    "organization_jif": f"I2I_JIF_ORGANIZATION_NIP_{date_suffix}.CSV",
    "policy_jif": f"I2I_JIF_POLICY_NIP_{date_suffix}.CSV",
    "premium_jif": f"I2I_JIF_PREMIUM_NIP_{date_suffix}.CSV",
    "claims_prog": f"I2I_Programs_CLAIM_NIP_{date_suffix}.CSV",
    "exposures_prog": f"I2I_Programs_EXPOSURE_NIP_{date_suffix}.CSV",
    "organization_prog": f"I2I_Programs_ORGANIZATION_NIP_{date_suffix}.CSV",
    "policy_prog": f"I2I_Programs_POLICY_NIP_{date_suffix}.CSV",
    "premium_prog": f"I2I_Programs_PREMIUM_NIP_{date_suffix}.CSV"
}

table_name = "claims"
file_name = map.get(table_name)

files = dbutils.fs.ls("/mnt/sas")

print("default")
if any(file_info.name == file_name for file_info in files):
    # return the full file path if the file exists
    print(f"/mnt/sas/{file_name}") # Use print for demo only
else:
    print("jif")
    file_name = map.get(f"{table_name}_jif")
    files = dbutils.fs.ls("/mnt/sas")

    if any(file_info.name == file_name for file_info in files):
        # return the full file path if the file exists
        print(f"/mnt/sas/{file_name}") # Use print for demo only
    else:
        print("prog")
        file_name = map.get(f"{table_name}_prog")
        files = dbutils.fs.ls("/mnt/sas")

        if any(file_info.name == file_name for file_info in files):
            # return the full file path if the file exists
            print(f"/mnt/sas/{file_name}") # Use print for demo only
        else:
            # Last iteration, return None if the file does not exist
            print("File not found")